In [ ]:
# importing packages
import pandas as pd
import numpy as np
import random as rnd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

#machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

Acquire Data ->
This data is available in Kaggle website and Pandas packages helps us to work with the datasets. We also combine these datasets to run certain operations on both datasets together.

In [ ]:
# loading dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine =[train_df, test_df]

# checking the size of both datasets
print('Size of the train dataset', train_df.shape)
print('Size of the test dataset', test_df.shape)

Analyze by describing data ->
To get to know the data better we check for what features are available in our dataset. We can then directly manipulate them and use them in our model.

In [ ]:
# data description
train_df.columns.unique()

Which features are Categorical? 


The values classify the smaples into set of similar samples. Within categorical features are the values nominal, ordinal, ratio or interval based?  Among ohter things this helps us select the appropirage plot for visualization.

 Categorical: Survived, Sex and Embarker.  Ordinal: Pclass.

Which features are Numerical?  

These value changes from sample to sample. Within numerical features are the values discrete, continuous or timeseries based. 

Continuous: Age and Fair. Discrete: SibSp and Parch.


In [ ]:
train_df.head(10)

##### What features are mixed data tyep?

Numerical, alphanumerical data are within same feature. These are candidates for correcting goals.

Ticket is a mix of numeric and alphanumeric data type. Cabin is a alphanumeric.

##### Which feature may contains data errors and typos?

Name feature may contain errors or typos as there are several ways to describe a name including titles, round brackets, and quotes are used for alternative or short names.

##### What are the data types for various features?

Several features are integers and many are floats.

In [ ]:
train_df.info()
print(['-'*40])
test_df.info()

##### Which feature contain null, empty or blank values?

In [ ]:
# This is code to check the missing data within our dataset
total = train_df.isnull().sum()
percentage = (train_df.isnull().sum()/train_df.count())*100
missing_data = pd.concat([total, percentage], axis=1, keys=['Total', 'Percentage']).sort_values(by ='Total', ascending=False)
missing_data

#### What is the distribution of values across the samples?

This helps us to determine, among other early insights, how representative is our training dataset of the actual problem domain.

Total samples are 891 or 40% of the actual number of the passenger on board (2224)

Survived is a categorical feature with values 0 and 1

Around 38% samples survived which is a representative of the actual survival rate at 32%

Most passengers (>75%) did not travel with parents or childern

Nearly 30% of the passengers had siblings/spouse abord

Fares varied significanlty for few passengers (<1%) paying as high as $512

Few elderly passengers(<1%) withing range 65-80


In [ ]:
train_df.describe()

##### What is the distribution of the categorical features

Names are unique across the dataset

Sex has two possible values with 65% male (top -male, freq -577)

Cabin values has several duplicates across the samples since some passengers shared cabin.

Embarked take three possible values. S port used by many passengers

Ticket value has high ratio (22%) of duplicates

In [ ]:
train_df.describe(include=['O'])

##### Analyze by pivoting features

To confirm some of our observations and assumptions, we can quickly analyze for feature correlation by pivoting features against each other. We can only do so at this stage for features which do not have any empty values. It also make sense doing so for features which are categorical (Sex), discrete(SibSp, Parch) and ordinal (Pclass)

Pclass - We observe significant correlation (>0.5) among Pclass-1 and Survived. We decided to include this feature in our model.

Sex - We confirm our observation during problem defination that feature Sex-female had very high survival rate at 74%.

SibSp & Parch - These features has zero correlation for certain values. It may be best to derive a feature or set of features from these individual features.

In [ ]:
train_df[['Survived', 'Sex']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[['Survived', 'Pclass']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[['Survived', 'SibSp']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[['Survived', 'Parch']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
# heatmap for correlation of the variables
cormat = train_df.corr()
f, ax =plt.subplots(figsize = (4, 4))
sns.heatmap(cormat, vmax=0.9, square=True)

#### Analyze by visualizing data

Correlating Numerical features

A histogram chart is used for analyzing continuous numerical variables like Age where banding or ranges will help identify useful patterns. The histogram can indicate distrbution of samples using automatically defined bins or equally ranged bands. This help us answer questions relating to specific bands (Did infants have better survival rate?)

Note: The x-axis in histogram visualizations represents the output of samples or passengers


Observations
- Infants (<4) had high survival rate
- Old passengers (65-80) survived
- Large number of 15-25 did not survive
- Most passengers are in 15-35 range

Decision
- We should consider Age in our model training
- Complete the Age feature for null values
- We should band age groups

In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins =20)

##### Correlating numerical and ordinal features

We can combine multiple features for identifying correlations using a single plot. This can be done with numerical and categorical features which have numeric values.

Observations:

- Pclass-3 had most passengers, however most did not survive.

- Infant passengers in Pclass-2 and Pclass-3 mostly survived.

- Most passengers in Pclass-1 survived. 

- Pclass varies in terms of Age distribution of passengers

Decision:
 
- Consider Pclass for model training

In [ ]:
grid = sns.FacetGrid(train_df, col = 'Survived', row = 'Pclass', height=2.2, aspect=1.2)
grid.map(plt.hist, 'Age', alpha=0.5, bins = 20)
grid.add_legend()

##### Correlating categorical features

Observations: 
- Female passengers had much better survival rate than males.

- Exception in Embarker=C where male had higher survival rate. This could be a correlation between Pclass and Embarked and in turn Pclass and Survived, not necessarily direct correlation between Embarked and Survived.

- Males had better survival rate in Pclass-3 when compared with Pclass-2 for C and Q ports.

- Ports of embarkation have varying survival rate for Pclass-3 and among male passengers.

Decision: 

- Add Sex feature to model training

- Complete and add Embarked feature to model training.

In [ ]:
grid = sns.FacetGrid(train_df, row='Embarked', height=1.8, aspect=1.4)
grid.map(sns.pointplot,'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

##### Correlating categorical amd numerical features

We may also want to correlate categorical features (with non-numeric values) and numeric features. We can consider correlating Embarked (categorical non-numeric), Sex (categorical non-numeric), Fare (numeric continuous), with Survived (categorical numeric).

Observations: 

- Higher Fare paying passengers had better survival rate.

- Port of Embarkations correlates with survival rate.

Decision
- Consider banding Fare feature

In [ ]:
grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', height=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=0.5, ci=None)
grid.add_legend()

##### Wrangle Data

Correcting by dropping features

In [ ]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

print("After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

Creating a new feature from existing feature

In the following code we extract new feature Title using regular expression. The RegExp pattern ('\w+\.') matches the first word with ends with the dot character within Name feature. The expand=False flag returns the DataFrame.

Observations:When we plot Title, Age and Survived, we not the following observations

- Most title band Age groups accurately. For example, Master title has Age mean of 5 years.

- Survival among Title Age bands varies slightly.

- Certain title mostly survived (Mme, Lady, Sir) or did not (Don, Rev, Jonkheer)

Decision:

- We decided to retain the new title feature for model training

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

In [ ]:
pd.crosstab(train_df['Title'], train_df['Survived'])

We can replace many titles with a more common name or classify them as Rare

In [ ]:
# replacing all th rare title from the dataset and adding theor outn 
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

We can convert the categorical title to ordinal

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

Now we can safely drop the Name feature from the training and testing dataset. We also do not need the PassengerId feature in the training dataset.

In [ ]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

##### Converting categorical feature

Now we can convert feature which contain strigs to numerical feature. This is required by most model algorihtm. Doing so will also help us achieving the completing goal.

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

##### Completing a numerical contiuous feature

We can consider three methods to complete a numerical contiuous feature

- A simple way is to generate random number between mean and standard deviation

- More accurate way to guess missing value is to use other correlated feature. In our case we note correlation among Age, Gender and Pclass. Guess Age values using median values for Age across set of Pclass and Gender feature combination. So meadian Age for Pclass = 1 and Gender=0, Pclass=1 and Gender=1 and so on

- Combine method 1 and 2. So instead of guessing age values based on meadian, use random numbers between median and standard deviation, based on sets of Pclass and Gender combination

Method 1 and 3 will introduce random noise in our model. The result from multiple execution may vary. We will prefer method 2 for our model

In [ ]:
grid = sns.FacetGrid(train_df, row='Pclass', col = 'Sex', height=1.8, aspect=1.2)
grid.map(plt.hist, 'Age', alpha= 0.5, bins = 20)
grid.add_legend()

In [ ]:
guess_ages = np.zeros((2, 3))
guess_ages

np.array([[0., 0., 0.], [0.,0.,0.]])

Now we iterate over Sex(0,1) and Pclass (1,2,3) to calculate guessed values of age for six combinations

In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

In [ ]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()

We can now remove the AgeBand feature

In [ ]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

Creating new feature combining existing features

We can create a new feature for FamilySize which combines Parch and SibSp. This will enable us to drop Parch and SibSp from our dataset

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

We can create another feature called IsAlone

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

Now we can drop Parch, SibSp and FamilySize from our dataset

In [ ]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

We can also create an artificial feature combining Pclass and Age

In [ ]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

##### Completing a Categorical feature

Embarked feature takes S,Q,C values based on port of embarkation. Our dataset missing two values. We fill these missing values based on most common occurance 

In [ ]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Quickly converting and completing the numeric feature

We can now quickly complete the Fare featue for single missing value in our testing dataset using mode to get the value tha occurs most frequently for this feature. We do this in one line of code.

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

In [ ]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

In [ ]:
test_df.head(10)

#### Model,Predict and Solve

Now we are ready to train a model and predict the required solution. There are 60+ predictive modelling algorithms to choose from. We must understand the type of problem and solution requirement to narrow down to a select few models which we can evaluate. Our problem is a classification and regression problem. We want to identify relationship between output (Survived or not) with other variables or features (Gender, Age, Port...). We are also perfoming a category of machine learning which is called supervised learning as we are training our model with a given dataset. With these two criteria - Supervised Learning plus Classification and Regression, we can narrow down our choice of models to a few. These include:

- Logistic Regression

- KNN or k-Nearest Neighbors
 
- Support Vector Machines

- Naive Bayes classifier

- Decision Tree

- Random Forrest

- Perceptron

- Artificial neural network

- RVM or Relevance Vector Machine

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

Logistic Regression is a useful model to run early in the workflow. Logistic regression measures the relationship between the categorical dependent variable (feature) and one or more independent variables (features) by estimating probabilities using a logistic function, which is the cumulative logistic distribution.

Note the confidence score generated by the model based on our training dataset.

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

We can use Logistic Regression to validate our assumptions and decisions for feature creating and completing goals. This can be done by calculating the coefficient of the features in the decision function.

Positive coefficients increase the log-odds of the response (and thus increase the probability), and negative coefficients decrease the log-odds of the response (and thus decrease the probability).

- Sex is highest positivie coefficient, implying as the Sex value increases (male: 0 to female: 1), the probability of Survived=1 increases the most.

- Inversely as Pclass increases, probability of Survived=1 decreases the most.

- This way Age*Class is a good artificial feature to model as it has second highest negative correlation with Survived.

- So is Title as second highest positive correlation.

In [ ]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

Next we model using Support Vector Machines which are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. Given a set of training samples, each marked as belonging to one or the other of two categories, an SVM training algorithm builds a model that assigns new test samples to one category or the other, making it a non-probabilistic binary linear classifier.

Note that the model generates a confidence score which is higher than Logistics Regression model.

In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

In pattern recognition, the k-Nearest Neighbors algorithm (or k-NN for short) is a non-parametric method used for classification and regression. A sample is classified by a majority vote of its neighbors, with the sample being assigned to the class most common among its k nearest neighbors (k is a positive integer, typically small). If k = 1, then the object is simply assigned to the class of that single nearest neighbor.

KNN confidence score is better than Logistics Regression but worse than SVM.

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

In machine learning, naive Bayes classifiers are a family of simple probabilistic classifiers based on applying Bayes' theorem with strong (naive) independence assumptions between the features. Naive Bayes classifiers are highly scalable, requiring a number of parameters linear in the number of variables (features) in a learning problem.

The model generated confidence score is the lowest among the models evaluated so far.

In [ ]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

The perceptron is an algorithm for supervised learning of binary classifiers (functions that can decide whether an input, represented by a vector of numbers, belongs to some specific class or not). It is a type of linear classifier, i.e. a classification algorithm that makes its predictions based on a linear predictor function combining a set of weights with the feature vector. The algorithm allows for online learning, in that it processes elements in the training set one at a time.

In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

This model uses a decision tree as a predictive model which maps features (tree branches) to conclusions about the target value (tree leaves). Tree models where the target variable can take a finite set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees. 

The model confidence score is the highest among models evaluated so far.

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

The next model Random Forests is one of the most popular. Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees (n_estimators=100) at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees.

The model confidence score is the highest among models evaluated so far. We decide to use this model's output (Y_pred) for creating our competition submission of results.

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

##### Model evaluation
We can now rank our evaluation of all the models to choose the best one for our problem. While both Decision Tree and Random Forest score the same, we choose to use Random Forest as they correct for decision trees' habit of overfitting to their training set.

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })

In [ ]:
submission.to_csv('submission.csv', index=False)